# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(42)
initial_params = np.random.random([15])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Fetch Dataset

In [3]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

# Import dataset

In [4]:
examples = mnist.data
classes = mnist.target

x = []
y = []
for (example, label) in zip(examples, classes):
    if label in ["0", "2", "4", "6", "8"]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(params[2], wires=2)
    qml.RY(params[3], wires=3)
    qml.CNOT(wires=[2, 3])
    qml.RY(params[4], wires=4)
    qml.RY(params[5], wires=5)
    qml.CNOT(wires=[5, 4])
    qml.RY(params[6], wires=6)
    qml.RY(params[7], wires=7)
    qml.CNOT(wires=[7, 6])

    # Second layer
    qml.RY(params[8], wires=1)
    qml.RY(params[9], wires=2)
    qml.CNOT(wires=[1, 2])
    qml.RY(params[10], wires=5)
    qml.RY(params[11], wires=6)
    qml.CNOT(wires=[6, 5])

    # Third layer
    qml.RY(params[12], wires=2)
    qml.RY(params[13], wires=5)
    qml.CNOT(wires=[2, 5])

    # Fourth layer
    qml.RY(params[14], wires=5)

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497]

Example features: [ 3.38764876 -1.94605776  0.49431788  0.86600787  0.48087724 -0.11930587
 -0.66084286  2.41654873]

Expectation value: -0.005924716389768325

 0: ──RY(3.39)────RY(0.375)───╭C────────────────────────────────────────────┤     
 1: ──RY(-1.95)───RY(0.951)───╰X──RY(0.601)───╭C────────────────────────────┤     
 2: ──RY(0.494)───RY(0.732)───╭C──RY(0.708)───╰X──RY(0.832)──╭C─────────────┤     
 3: ──RY(0.866)───RY(0.599)───╰X─────────────────────────────│──────────────┤     
 4: ──RY(0.481)───RY(0.156)───╭X─────────────────────────────│──────────────┤     
 5: ──RY(-0.119)──RY(0.156)───╰C──RY(0.0206)──╭X──RY(0.212)──╰X──RY(0.182)──┤ ⟨Z⟩ 
 6: ──RY(-0.661)──RY(0.0581)──╭X──RY(0.97)────╰C────────────────────────────┤     
 7: ──RY(2.42)────RY(0.866)───╰C──────────────────────────────────────

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.5270108043217288
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.5471788715486194
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.5882352941176471
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.6151260504201681
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.6211284513805522
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.6012004801920768
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.624249699879952
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.6249699879951981
best acc

epoch 341
epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.6292917166866747
test_accuracies: [0.624249699879952, 0.6249699879951981, 0.6175270108043217, 0.6148859543817526, 0.6230492196878752, 0.6218487394957983, 0.6309723889555823, 0.6297719087635054, 0.6343337334933974, 0.6240096038415366, 0.624249699879952, 0.6252100840336134, 0.6244897959183673, 0.6225690276110445, 0.6261704681872748, 0.6211284513805522, 0.6259303721488596, 0.6249699879951981, 0.6218487394957983, 0.6204081632653061, 0.6302521008403361, 0.6302521008403361, 0.6266506602641057, 0.6247298919567827, 0.6312124849939976, 0.6009603841536615, 0.6139255702280912, 0.6259303721488596, 0.6333733493397359, 0.6292917166866747]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.6338535414165667
test_accuracies: [0.6249699879951981, 0.6175270108043217, 0.6148859543817526, 0.6230492196878752, 0.6218487394957983, 0.630972

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [-0.02579116  1.38752914  0.49615807  1.84297505 -0.09992936  0.44666962
  1.01945083  1.40061945  0.60451609  0.5563285  -0.67761508  0.79164729
  0.38770704  0.94882035  0.66965521]


# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.6224761904761904


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 68.95486950874329
training time: 4443.956442832947
test time: 264.7453784942627
total time: 4777.656690835953
